# This is the Object Oriented Clean version of the Cross Mapper.

In [ ]:
!pip install gdown

import gdown

url = 'https://drive.google.com/file/d/1-5fOSHOSB9UXyP_enOoZNAMScrePVcMD/view'

file_id = url.split('/d/')[1].split('/')[0]
download_url = f'https://drive.google.com/uc?id={file_id}'

gdown.download(download_url, output='best.pt', quiet=False)

!pip install uv
!uv pip install -r req.txt

In [ ]:
import os
import cv2
import numpy as np
import torch
import supervision as sv
from ultralytics import YOLO
from collections import defaultdict
import json
from typing import Union, List, Dict, Tuple

TrackingDataType = Dict[
    int,  
    Dict[
        int,  
        Dict[
            int,  
            List[Union[List[float], Tuple[float, float, float, float]]]  # bounding boxes
        ]
    ]
]

class ObjectDetector:
    def __init__(self, model_path='best.pt'):
        self.model_path = model_path
        self.model = None
        self.device = self._get_device()
        self._load_model()

    def _get_device(self):
        if torch.backends.mps.is_available():
            return torch.device("mps") 
        elif torch.cuda.is_available():
            return torch.device("cuda")
        else:
            return torch.device("cpu")

    def _load_model(self):
        print(f"Loading model from: {self.model_path}")
        self.model = YOLO(self.model_path)
        self.model.to(self.device)
        print(f"Using device: {self.device}")
        print("Model loaded successfully!")

    def detect_and_annotate(self, video_path: str, camera_name: str) -> None:
        print(f"\nStarting detection for {camera_name} video: {video_path}")

        results_generator = self.model.predict(source=video_path, stream=True, conf=0.5, iou=0.7, classes=None, verbose=False)

        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            print(f"Error: Could not open video file {video_path}")
            return

        frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = int(cap.get(cv2.CAP_PROP_FPS))
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        print(f"Video {camera_name} properties: {frame_width}x{frame_height} @ {fps} FPS, {total_frames} frames")

        output_video_path = f'results/Detections-{os.path.basename(video_path)}'
        os.makedirs(os.path.dirname(output_video_path), exist_ok=True)

        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

        frame_count = 0
        for result in results_generator:
            frame_count += 1
            if frame_count % 100 == 0:
                print(f"  {camera_name} processing frame {frame_count}/{total_frames}...")

            frame = result.orig_img
            boxes = result.boxes

            for box in boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                confidence = float(box.conf[0])
                class_id = int(box.cls[0])
                class_name = self.model.names[class_id]

                color = (0, 255, 0) 
                if class_name == 'ball':
                    color = (0, 165, 255) 

                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                label = f"{class_name} {confidence:.2f}"
                cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

            out.write(frame)

        cap.release()
        out.release()
        print(f"Finished processing : {camera_name} . output saved to: {output_video_path}")

class ObjectTracker:
    def __init__(self, model, tracker_config_path='custom_tracker-botsort-reid.yaml'):
        self.model = model
        self.tracker_config_path = tracker_config_path
        self._write_tracker_config()

        self.class_names = [x[0].upper() for y, x in model.names.items()]
        self.ellipse_annotator = sv.EllipseAnnotator(
            color=sv.ColorPalette.from_hex(['#00BFFF', '#FF1493', '#FFD700']),
            thickness=1
        )
        self.label_annotator = sv.LabelAnnotator(
            color=sv.ColorPalette.from_hex(['#00BFFF', '#FF1493', '#FFD700']),
            text_color=sv.Color.from_hex('#000000'),
            text_position=sv.Position.TOP_CENTER,
            text_thickness=0
        )

    def _write_tracker_config(self):
        custom_tracker_yaml_content = """
tracker_type: botsort
track_high_thresh: 0.45 # threshold for the first association
track_low_thresh: 0.15   # threshold for the second association
new_track_thresh: 0.4   # threshold for init new track if the detection does not match any tracks
track_buffer: 90 # Kept it a bit higher for long-term tracking
match_thresh: 0.9 # threshold for matching tracks
fuse_score: True  # Whether to fuse confidence scores with the iou distances before matching

# BoT-SORT settings
gmc_method: sparseOptFlow

# ReID model related thresh
proximity_thresh: 0.6 # minimum IoU for valid match with ReID
appearance_thresh: 0.6 # minimum appearance similarity for ReID
with_reid: True
model: auto
"""
        with open(self.tracker_config_path, 'w') as f:
            f.write(custom_tracker_yaml_content)
        print(f"Custom tracker config saved to: {self.tracker_config_path}")

    def track_and_annotate(self, video_path: str, camera_name: str) -> TrackingDataType:
        output_video_path = f'results/BotSort-Tracked-{camera_name}.mp4'

        cap = cv2.VideoCapture(video_path)
        fps = cap.get(cv2.CAP_PROP_FPS)
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fourcc = cv2.VideoWriter_fourcc(*'mp4v') 

        out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))
        print(f"Writing output to: {output_video_path}")

        tracked_data: TrackingDataType = {}
        frame_number = 0

        while cap.isOpened():
            success, frame = cap.read()
            if success:
                results = self.model.track(frame,
                                           tracker=self.tracker_config_path,
                                           stream=True,
                                           conf=0.52,
                                           iou=0.7,   # IoU threshold for detection NMS
                                           verbose=False,
                                           persist=True)

                frame_number += 1
                if frame_number % 25 == 0:
                    print(f"Processed {frame_number} frames")

                for result in results:
                    framer = result.orig_img
                    tracked_data[frame_number] = {}
                    detections = sv.Detections.from_ultralytics(result)

                    if result.boxes.id is not None:
                        all_detections = detections[detections.class_id != 0]

                        #if tracker id is assigned
                        if all_detections.tracker_id is not None:
                            labels = [
                                f"#{int(tracker_id)} {self.class_names[int(class_id)]}"
                                for tracker_id, class_id in zip(all_detections.tracker_id, all_detections.class_id)
                            ]

                            for class_id, tracker_id, box in zip(
                                all_detections.class_id,
                                all_detections.tracker_id,
                                all_detections.xyxy
                            ):
                                class_id = int(class_id)
                                tracker_id = int(tracker_id)
                                box = box.tolist()

                                #adding the data in our data structure
                                if class_id not in tracked_data[frame_number]:
                                    tracked_data[frame_number][class_id] = {}
                                tracked_data[frame_number][class_id][tracker_id] = box
                        else:
                            # when tracker_id is None
                            labels = ["NA" for class_id in all_detections.class_id]

                        # annotation on the original frame
                        annotated_framer = framer.copy()
                        annotated_framer = self.ellipse_annotator.annotate(
                            scene=annotated_framer,
                            detections=all_detections
                        )
                        annotated_framer = self.label_annotator.annotate(
                            scene=annotated_framer,
                            detections=all_detections,
                            labels=labels
                        )
                    else:
                        # If no tracks are found ( at the very start)
                        annotated_framer = framer.copy()
                        all_detections = detections[detections.class_id != 0]
                        all_detections = all_detections[all_detections.confidence > 0.51]
                        annotated_framer = self.ellipse_annotator.annotate(
                            scene=annotated_framer,
                            detections=all_detections
                        )
                        labels_no_track = ["NA" for class_id in all_detections.class_id]
                        annotated_framer = self.label_annotator.annotate(
                            scene=annotated_framer,
                            detections=all_detections,
                            labels=labels_no_track
                        )
                    out.write(annotated_framer)
            else:
                break

        cap.release()
        out.release()
        return tracked_data

class VideoCrossMapper:
    def __init__(self, object_detector: ObjectDetector):
        self.object_detector = object_detector
        self.sift = cv2.SIFT_create()
        # FLANN parameters
        FLANN_INDEX_KDTREE = 1
        index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
        search_params = dict(checks=50)
        self.flann = cv2.FlannBasedMatcher(index_params, search_params)
        self.consistent_object_mapping = {}
        #config
        self.MIN_MATCH_COUNT = 10  # number of good matches required to find a homography
        self.SIFT_RATIO_TEST_THRESHOLD = 0.75 # (0.7-0.8 is standard)
        self.RANSAC_REPROJECTION_THRESHOLD = 5.0 # maximum pixel distance for a point to be an inlier
        self.MIN_IOU_THRESHOLD = 0.1

        self.ellipse_annotator_final = sv.EllipseAnnotator(
            color=sv.ColorPalette.from_hex(["#000000", "#000000", "#2E2704"]),
            thickness=1
        )
        self.label_annotator_final = sv.LabelAnnotator(
            color=sv.ColorPalette.from_hex(['#00BFFF', '#FF1493', '#FFD700']),
            text_color=sv.Color.from_hex('#000000'),
            text_position=sv.Position.TOP_CENTER,
            text_thickness=0
        )

    def _get_frame(self, video_path: str, frame_number: int):
        cap = cv2.VideoCapture(video_path)
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
        ret, frame = cap.read()
        cap.release()
        if not ret:
            print(f"Warning: Could not read frame {frame_number} from {video_path}")
            return None
        return frame

    #transforms a single bounding box using homography
    def _transform_bbox(self, bbox: List[float], H: np.ndarray) -> List[float]:
        x1, y1, x2, y2 = bbox
        corners = np.array([
            [x1, y1],
            [x2, y1],
            [x2, y2],
            [x1, y2]
        ], dtype=np.float32).reshape(-1, 1, 2) # reshape for perspectiveTransform

        # apply the homography
        transformed_corners = cv2.perspectiveTransform(corners, H)

        # calculating the new bounding box from the transformed corners
        x_new = transformed_corners[:, 0, 0]
        y_new = transformed_corners[:, 0, 1]

        new_x1 = float(min(x_new))
        new_y1 = float(min(y_new))
        new_x2 = float(max(x_new))
        new_y2 = float(max(y_new))

        return [new_x1, new_y1, new_x2, new_y2]

    def _calculate_iou(self, boxA: List[float], boxB: List[float]) -> float:
        xA = max(boxA[0], boxB[0])
        yA = max(boxA[1], boxB[1])
        xB = min(boxA[2], boxB[2])
        yB = min(boxA[3], boxB[3])

        # compute the area of intersection rectangle
        interArea = max(0, xB - xA) * max(0, yB - yA)

        # compute the area of both the prediction and ground-truth rectangles
        boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
        boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])

        # edge case where boxes have zero area
        if boxAArea <= 0 or boxBArea <= 0:
            return 0.0

        # compute the intersection over union
        unionArea = float(boxAArea + boxBArea - interArea)
        if unionArea <= 0:
            return 0.0

        iou = interArea / unionArea
        return iou

    def perform_cross_mapping(self, video1_path: str, video2_path: str,
                              tracked_data_v1: TrackingDataType,
                              tracked_data_v2: TrackingDataType) -> Dict:

        frame_numbers_v1 = list(tracked_data_v1.keys()) if tracked_data_v1 else [0]
        frame_numbers_v2 = list(tracked_data_v2.keys()) if tracked_data_v2 else [0]
        max_frames = max(max(frame_numbers_v1), max(frame_numbers_v2)) + 1

        for frame_num in range(max_frames):
            print(f"\nProcessing Frame {frame_num}...")

            # Skip if no tracking data for this frame in both results
            if frame_num not in tracked_data_v1 and frame_num not in tracked_data_v2:
                print(f"No tracking data for frame {frame_num}, skipping.")
                self.consistent_object_mapping[frame_num] = {}
                continue

            frame1 = self._get_frame(video1_path, frame_num)
            frame2 = self._get_frame(video2_path, frame_num)

            if frame1 is None or frame2 is None:
                print(f"Skipping frame {frame_num}")
                self.consistent_object_mapping[frame_num] = {}
                continue

            # Convert frames to grayscale for SIFT
            gray1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
            gray2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)

            #SIFT Feature Detection and Description
            kp1, des1 = self.sift.detectAndCompute(gray1, None)
            kp2, des2 = self.sift.detectAndCompute(gray2, None)

            if des1 is None or des2 is None or len(kp1) < self.MIN_MATCH_COUNT or len(kp2) < self.MIN_MATCH_COUNT:
                print(f"Not enough SIFT features detected in frame {frame_num} for homography calculation.")
                self.consistent_object_mapping[frame_num] = {} # Store an empty entry
                continue

            # FLANN-based feature matching with ratio test
            try:
                matches = self.flann.knnMatch(des1, des2, k=2)
            except cv2.error as e:
                print(f"Error during feature matching in frame {frame_num}: {e}")
                self.consistent_object_mapping[frame_num] = {}
                continue

            #Lowe's ratio test to filter good matches
            good_matches = []
            for match_pair in matches:
                if len(match_pair) == 2:  # ensuring we have 2 matches
                    m, n = match_pair
                    if m.distance < self.SIFT_RATIO_TEST_THRESHOLD * n.distance:
                        good_matches.append(m)

            if len(good_matches) < self.MIN_MATCH_COUNT:
                print(f"Not enough good SIFT matches ({len(good_matches)}) in frame {frame_num} for homography calculation.")
                self.consistent_object_mapping[frame_num] = {} # Store an empty entry
                continue

            src_pts = np.float32([kp1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
            dst_pts = np.float32([kp2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

            # calculating homography matrix H using ransac
            H, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, self.RANSAC_REPROJECTION_THRESHOLD)

            if H is None:
                print(f"Could not calculate homography for frame {frame_num}.")
                self.consistent_object_mapping[frame_num] = {} # Store an empty entry
                continue

            print(f"Homography calculated for frame {frame_num} with {len(good_matches)} matches")

            current_frame_mapping = {}

            v1_frame_data = tracked_data_v1.get(frame_num, {})
            v2_frame_data = tracked_data_v2.get(frame_num, {})

            for class_id_v1, tracking_ids_v1 in v1_frame_data.items():
                if class_id_v1 not in current_frame_mapping:
                    current_frame_mapping[class_id_v1] = {}

                for tracking_id_v1, bbox_data_v1 in tracking_ids_v1.items():
                    if isinstance(bbox_data_v1, list):
                        if len(bbox_data_v1) == 4 and all(isinstance(x, (int, float)) for x in bbox_data_v1):
                            # bbox format [x1,y1,x2,y2]
                            bbox_v1 = bbox_data_v1
                        else:
                            print(f"Invalid bbox format for tracking_id {tracking_id_v1}")
                            continue
                    else:
                        print(f"error for tracking_id {tracking_id_v1}")
                        continue

                    try:
                        transformed_bbox_v2 = self._transform_bbox(bbox_v1, H)
                    except Exception as e:
                        print(f"Error transforming bbox: {e}")
                        continue

                    best_iou = self.MIN_IOU_THRESHOLD
                    best_match_info = None

                    for class_id_v2, tracking_ids_v2 in v2_frame_data.items():
                        #matching only same class_ids
                        if class_id_v1 != class_id_v2:
                            continue

                        for tracking_id_v2, bbox_data_v2 in tracking_ids_v2.items():
                            # Handle different bbox formats for video 2
                            if isinstance(bbox_data_v2, list):
                                if len(bbox_data_v2) == 4 and all(isinstance(x, (int, float)) for x in bbox_data_v2):
                                    bbox_v2_detected = bbox_data_v2
                                else:
                                    continue
                            else:
                                continue

                            iou = self._calculate_iou(transformed_bbox_v2, bbox_v2_detected)

                            if iou > best_iou:
                                best_iou = iou
                                best_match_info = {
                                    'class_id': class_id_v2,
                                    'tracking_id': tracking_id_v2,
                                    'bbox': bbox_v2_detected,
                                    'iou': iou
                                }

                    # updating the data structure
                    object_mapping_entry = {
                        'video1_bbox': bbox_v1,
                        'video2_transformed_bbox': transformed_bbox_v2,
                        'video2_matched_detection': best_match_info # will be None if no good match
                    }

                    current_frame_mapping[class_id_v1][tracking_id_v1] = object_mapping_entry

            self.consistent_object_mapping[frame_num] = current_frame_mapping

        print(f"Cross Mapping Successful!")
        return self.consistent_object_mapping

    def generate_consistent_id_mapping(self, consistent_object_mapping: Dict):
        tracker_mapping_counts = {}  # {(class_id, track_id_v1, track_id_v2): count}
        for frame_num, frame_data in consistent_object_mapping.items():
            for class_id, tracks in frame_data.items():
                for track_id_v1, mapping in tracks.items():
                    match_info = mapping.get('video2_matched_detection')
                    if match_info is not None:
                        track_id_v2 = match_info['tracking_id']
                        key = (class_id, track_id_v1, track_id_v2)
                        tracker_mapping_counts[key] = tracker_mapping_counts.get(key, 0) + 1

        sorted_mappings = sorted(tracker_mapping_counts.items(), key=lambda x: x[1], reverse=True)

        # This dictionary will map a tacticam_id to its most likely broadcast_id.
        tacticam_to_broadcast_id_map = {}

        broadcast_ids_mapped = set()
        tacticam_ids_mapped = set()

        for (class_id, b_id, t_id), count in sorted_mappings:
            if b_id not in broadcast_ids_mapped and t_id not in tacticam_ids_mapped:
                tacticam_to_broadcast_id_map[t_id] = b_id
                broadcast_ids_mapped.add(b_id)
                tacticam_ids_mapped.add(t_id)

        print("Created a stable global mapping for consistent IDs.")
        print(f"Total stable mappings created: {len(tacticam_to_broadcast_id_map)}")

        simplified_mappings = {}
        for frame_num, frame_data in consistent_object_mapping.items():
            simplified_mappings[frame_num] = {}
            for class_id, tracks in frame_data.items():
                for track_id_v1, mapping in tracks.items():
                    match_info = mapping.get('video2_matched_detection')
                    if match_info is not None:
                        if class_id not in simplified_mappings[frame_num]:
                            simplified_mappings[frame_num][class_id] = {}
                        simplified_mappings[frame_num][class_id][track_id_v1] = {
                            'tacticam_id': match_info['tracking_id'],
                            'iou': match_info['iou']
                        }

        MAPPINGS_FILE = 'tracker_id_mappings.json'
        with open(MAPPINGS_FILE, 'w') as f:
            json.dump(simplified_mappings, f, indent=2)
        print("\nTracker mappings saved to 'tracker_id_mappings.json'")

        return tacticam_to_broadcast_id_map

    def annotate_with_consistent_ids(self, input_video_path: str,
                                     output_video_name: str,
                                     tracked_data: TrackingDataType,
                                     is_broadcast_video: bool,
                                     tacticam_to_broadcast_id_map: Dict):

        output_path = f"results/{output_video_name}"
        print(f"\n Starting consistent annotation for: {input_video_path}")

        cap = cv2.VideoCapture(input_video_path)
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = int(cap.get(cv2.CAP_PROP_FPS))

        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

        frame_num = 0
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            frame_num += 1

            detections_in_frame = tracked_data.get(frame_num, {})

            if not detections_in_frame:
                out.write(frame) # original frame if no detections
                continue

            all_bboxes = []
            all_class_ids = []
            all_tracker_ids = []

            for class_id, tracks in detections_in_frame.items():
                for tracker_id, bbox in tracks.items():
                    all_bboxes.append(bbox)
                    all_class_ids.append(class_id)
                    all_tracker_ids.append(tracker_id)

            # Convert to supervision Detections object
            sv_detections = sv.Detections(
                xyxy=np.array(all_bboxes),
                class_id=np.array(all_class_ids),
                tracker_id=np.array(all_tracker_ids)
            )

            # Generate consistent labels
            labels = []
            for tracker_id, class_id in zip(sv_detections.tracker_id, sv_detections.class_id):
                class_name = self.object_detector.model.names[class_id][0].upper()

                if is_broadcast_video:
                    #  broadcast ID is the master ID
                    final_id = tracker_id
                else:
                    #its corresponding master (broadcast) ID
                    final_id = tacticam_to_broadcast_id_map.get(tracker_id, f"T{tracker_id}")

                labels.append(f"#{final_id} {class_name}")

            # Annotate the frame
            annotated_frame = frame.copy()
            annotated_frame = self.ellipse_annotator_final.annotate(
                scene=annotated_frame,
                detections=sv_detections
            )
            annotated_frame = self.label_annotator_final.annotate(
                scene=annotated_frame,
                detections=sv_detections,
                labels=labels
            )

            out.write(annotated_frame)

        cap.release()
        out.release()
        print(f"Consistently annotated video to: {output_path}")

    def combine_videos(self, video1_path: str, video2_path: str, output_path: str):
        broadcast_cap = cv2.VideoCapture(video1_path)
        tacticam_cap = cv2.VideoCapture(video2_path)

        width = int(broadcast_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(broadcast_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = broadcast_cap.get(cv2.CAP_PROP_FPS)

        # output width will be double the width
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_path, fourcc, fps, (width * 2, height))

        while True:
            ret1, frame1 = broadcast_cap.read()
            ret2, frame2 = tacticam_cap.read()

            if not ret1 or not ret2:
                break
            combined_frame = np.hstack((frame1, frame2))
            out.write(combined_frame)

        broadcast_cap.release()
        tacticam_cap.release()
        out.release()
        cv2.destroyAllWindows()
        print("Your videos have been consistently mapped. Your Output is Ready.")



In [ ]:
# Define video paths
broadcast_video_path = 'br-frame_matched.mp4'
tacticam_video_path = 'ta-frame_matched.mp4'

# Step 1: Object Detection
detector = ObjectDetector(model_path='best.pt')
print(f"Processing broadcast video: {broadcast_video_path}")
detector.detect_and_annotate(broadcast_video_path, "Broadcast")
print(f"Processing tacticam video: {tacticam_video_path}")
detector.detect_and_annotate(tacticam_video_path, "Tacticam")
cv2.destroyAllWindows()
print("\nAll video processing tasks completed.")

# Step 2: Object Tracking
tracker = ObjectTracker(model=detector.model)
broadcast_tracked_players = tracker.track_and_annotate(broadcast_video_path, "Broadcast")
tacticam_tracked_players = tracker.track_and_annotate(tacticam_video_path, "Tacticam")
print("Video processing complete. Output saved.")

# Step 3: Video Cross Mapping and Consistent Annotation
mapper = VideoCrossMapper(object_detector=detector)
consistent_object_mapping = mapper.perform_cross_mapping(
    broadcast_video_path, tacticam_video_path,
    broadcast_tracked_players, tacticam_tracked_players
)

tacticam_to_broadcast_id_map = mapper.generate_consistent_id_mapping(consistent_object_mapping)

mapper.annotate_with_consistent_ids(
    input_video_path=broadcast_video_path,
    output_video_name="Final-Consistent-Broadcast.mp4",
    tracked_data=broadcast_tracked_players,
    is_broadcast_video=True,
    tacticam_to_broadcast_id_map=tacticam_to_broadcast_id_map
)

mapper.annotate_with_consistent_ids(
    input_video_path=tacticam_video_path,
    output_video_name="Final-Consistent-Tacticam.mp4",
    tracked_data=tacticam_tracked_players,
    is_broadcast_video=False,
    tacticam_to_broadcast_id_map=tacticam_to_broadcast_id_map
)
cv2.destroyAllWindows()
print("\n All videos have been generated again with consistent tracking IDs.")

# Step 4: Combine Final Videos
mapper.combine_videos(
    "results/Final-Consistent-Broadcast.mp4",
    "results/Final-Consistent-Tacticam.mp4",
    "results/Final_Result.mp4"
)
